# Top
Want to run hparam sweep on XGBoost

In [3]:
# google colab preliminaries
from google.colab import drive
drive.mount('/content/gdrive')
drive_path = "/content/gdrive/MyDrive/Career/ML Study/kaggle_ubiquant/"

!pip install git+https://github.com/dennischenfeng/kaggle-ubiquant.git

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Mounted at /content/gdrive


In [51]:
%load_ext autoreload
%autoreload 2

from kaggle_ubiquant.dataset import generate_dataset, DatasetConfig, Dataset
from kaggle_ubiquant.model import generate_model, ModelConfig
from kaggle_ubiquant.train import training_run
import pandas as pd
import numpy as np
from typing import Iterable, Dict, Tuple, Callable, Optional
from scipy.stats import pearsonr
import wandb
import dataclasses
import optuna
import pathlib

ROOT_DIR = pathlib.Path(drive_path)

In [6]:
df_smallest = pd.read_csv(ROOT_DIR / 'data/train_smallest.csv')
df_small = pd.read_csv(ROOT_DIR / 'data/train_small.csv')

In [66]:
def suggest_configs(
    trial: optuna.Trial, 
    n_investments_train: int,
    n_investments_test: int, 
    n_investments_overlap: int,
    gpu: bool = False,
    start_test_time_id: int = 900,
) -> Tuple[DatasetConfig, ModelConfig]:
    """
    Suggest parameters for DatasetConfig and ModelConfig
    """
    dc = DatasetConfig(
        n_investments_train, 
        n_investments_test, 
        n_investments_overlap,
        use_investment_id=trial.suggest_categorical('use_investment_id', [True, False]),
        start_test_time_id=start_test_time_id,
    )

    tree_method = 'gpu_hist' if gpu else None
    mc = ModelConfig(
        model_kwargs=dict(
            learning_rate=trial.suggest_categorical(
                'learning_rate',[0.05, 0.10, 0.15, 0.20, 0.25, 0.30]
            ),
            max_depth=trial.suggest_categorical(
                'max_depth', [3, 4, 5, 6, 8, 10, 12, 15]
            ),
            min_child_weight=trial.suggest_categorical(
                'min_child_weight', [1, 3, 5, 7]
            ),
            gamma=trial.suggest_categorical(
                'gamma', [0.0, 0.1, 0.2 , 0.3, 0.4]
            ),
            colsample_bytree=trial.suggest_categorical(
                'colsample_bytree', [ 0.3, 0.4, 0.5 , 0.7 ]
            ),
            tree_method=tree_method,
        )
    )

    return dc, mc

In [30]:
# test
study = optuna.create_study(direction='maximize')
trial = study.ask()
dc, mc = suggest_configs(trial, 5, 5, 5, gpu=True)

[I 2022-02-18 02:36:48,259] A new study created in memory with name: no-name-c5fc7bb9-99ea-4eea-a8c9-82d73a9cca98


# Test simplest case

In [24]:
study = optuna.create_study(direction='maximize')
for i in range(3):
    trial = study.ask()
    
    # if i == 0:
    #     dc = DatasetConfig(7, 5, 5)
    #     mc = ModelConfig()
    # else:
    dc, mc = suggest_configs(trial, 7, 5, 5)
    
    scores = []
    for j in range(2):
        scores.append(training_run(df_smallest, dc, mc, 'kaggleUbiquant-scratch'))
        
    study.tell(trial, np.mean(scores))

[I 2022-02-18 01:54:05,727] A new study created in memory with name: no-name-6e45a9b8-64a5-42eb-a5e2-a108bcb975cf
100%|██████████| 4940/4940 [00:00<00:00, 31253.23it/s]
/usr/local/lib/python3.7/dist-packages/kaggle_ubiquant/dataset.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target_lag1'] = compute_lag1(df, lag_default_value=dc.lag_default_value)
100%|██████████| 1547/1547 [00:00<00:00, 29026.39it/s]
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


pearsonr,▁
pearsonr,0.11417


100%|██████████| 4904/4904 [00:00<00:00, 29855.17it/s]
/usr/local/lib/python3.7/dist-packages/kaggle_ubiquant/dataset.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target_lag1'] = compute_lag1(df, lag_default_value=dc.lag_default_value)
100%|██████████| 1579/1579 [00:00<00:00, 31373.13it/s]
wandb: Currently logged in as: dennisfeng (use `wandb login --relogin` to force relogin)


...

pearsonr,▁
pearsonr,0.02191


100%|██████████| 1417/1417 [00:00<00:00, 29092.89it/s]


pearsonr,▁
pearsonr,0.01236


100%|██████████| 1457/1457 [00:00<00:00, 31724.55it/s]


pearsonr,▁
pearsonr,0.05727


# Test df_small, using gpu, 100% overlap

How many investment_ids are available for training in df_small? Requires time_id < 900

In [27]:
num_train_iid = len(pd.unique(df_small[df_small.time_id < 900].investment_id))
num_train_iid

272

In [28]:
study = optuna.create_study(direction='maximize')
for i in range(3):
    trial = study.ask()
    
    if i == 0:
        dc = DatasetConfig(num_train_iid, num_train_iid, num_train_iid)
        mc = ModelConfig(model_kwargs=dict(tree_method='gpu_hist'))
    else:
        dc, mc = suggest_configs(
            trial, num_train_iid, num_train_iid, num_train_iid, gpu=True
        )
    
    scores = []
    for j in range(2):
        scores.append(training_run(df_small, dc, mc, 'kaggleUbiquant-scratch'))
        
    study.tell(trial, np.mean(scores))

[I 2022-02-18 01:58:15,794] A new study created in memory with name: no-name-3e9a19de-96e1-40f8-a23b-1ec4b5663dc2
100%|██████████| 173178/173178 [00:06<00:00, 25369.29it/s]
/usr/local/lib/python3.7/dist-packages/kaggle_ubiquant/dataset.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target_lag1'] = compute_lag1(df, lag_default_value=dc.lag_default_value)
100%|██████████| 78261/78261 [00:02<00:00, 31241.62it/s]


pearsonr,▁
pearsonr,0.07389


100%|██████████| 78261/78261 [00:02<00:00, 28211.35it/s]


pearsonr,▁
pearsonr,0.07389


100%|██████████| 78261/78261 [00:02<00:00, 30367.45it/s]


...

pearsonr,▁
pearsonr,0.10874


100%|██████████| 78261/78261 [00:02<00:00, 31041.14it/s]


pearsonr,▁
pearsonr,0.09725


100%|██████████| 78261/78261 [00:02<00:00, 30043.80it/s]


pearsonr,▁
pearsonr,0.09725


# Investigate: why correlation not as high as previous?

Differences:
* start_time_id for test was at 90% point

Try with 90% now.

In [37]:
# emulate 20220214.ipynb's start_test_time_id
start_test_time_id = df_small.iloc[
    int(0.90 * df_small.shape[0]), df_small.columns.get_loc('time_id')
]
start_test_time_id

1126

Looks like I need to compare start_test_time_id of 900 vs 1126.

## Use start_test_time_id = 900

In [60]:
start_test_time_id = 900

In [61]:
num_train_iid = len(pd.unique(
    df_small[df_small.time_id < start_test_time_id].investment_id
))

dc = DatasetConfig(
    num_train_iid, num_train_iid, num_train_iid, 
    start_test_time_id=start_test_time_id
)
mc = ModelConfig(model_kwargs=dict(tree_method='gpu_hist'))
training_run(df_small, dc, mc, 'kaggleUbiquant-investigateStartTime')


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.7/bdb.py", line 332, in set_trace
    sys.settrace(self.trace_dispatch)



> /usr/local/lib/python3.7/dist-packages/kaggle_ubiquant/dataset.py(38)generate_dataset()
-> dc = dataset_config
(Pdb) c



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.7/bdb.py", line 343, in set_continue
    sys.settrace(None)

100%|██████████| 173178/173178 [00:05<00:00, 31589.65it/s]
/usr/local/lib/python3.7/dist-packages/kaggle_ubiquant/dataset.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dc = dataset_config
100%|██████████| 78261/78261 [00:02<00:00, 30551.99it/s]


pearsonr,▁
pearsonr,0.0882


0.0882012760799752

## Use start_test_time_id = 1126

In [62]:
start_test_time_id = 1126

In [63]:
num_train_iid = len(pd.unique(
    df_small[df_small.time_id < start_test_time_id].investment_id
))

dc = DatasetConfig(
    num_train_iid, num_train_iid, num_train_iid, 
    start_test_time_id=start_test_time_id
)
mc = ModelConfig(model_kwargs=dict(tree_method='gpu_hist'))
training_run(df_small, dc, mc, 'kaggleUbiquant-investigateStartTime')

100%|██████████| 231530/231530 [00:07<00:00, 30890.03it/s]
/usr/local/lib/python3.7/dist-packages/kaggle_ubiquant/dataset.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if dc.num_lags == 1:
100%|██████████| 25644/25644 [00:00<00:00, 31103.56it/s]


pearsonr,▁
pearsonr,0.12389


0.12388756799144279

Conclusion: I think most of the improvement was due to the higher start_test_time_id, which was cheating a little bit, compared with what I think the true test data would be like.


# Run

I expect true test data to have a decent amount of new investment_id's. Best guess is that 75% of the test data will be investment_id's of the training data, but remaining will be new. We'll emulate that in this run

In [67]:
overlap_proportion = 0.75
num_test_iid = len(pd.unique(df_small.investment_id))
num_train_iid = int(overlap_proportion * num_test_iid)
print(num_test_iid, num_train_iid)

start_test_time_id = 900

300 225


In [68]:
study = optuna.create_study(direction='maximize')
for i in range(100):
    trial = study.ask()
    
    if i == 0:
        dc = DatasetConfig(num_train_iid, num_test_iid, num_train_iid, start_test_time_id=900)
        mc = ModelConfig(model_kwargs=dict(tree_method='gpu_hist'))
    else:
        dc, mc = suggest_configs(
            trial, num_train_iid, num_train_iid, num_train_iid, gpu=True, start_test_time_id=900
        )
    
    scores = []
    for j in range(3):
        scores.append(training_run(df_small, dc, mc, 'kaggleUbiquant-XGBoostHparamSearch'))
        
    study.tell(trial, np.mean(scores))

[I 2022-02-18 03:18:33,734] A new study created in memory with name: no-name-e6c90d94-3b67-4381-ad8f-62fb1c5f496e
100%|██████████| 141926/141926 [00:04<00:00, 29066.41it/s]
/usr/local/lib/python3.7/dist-packages/kaggle_ubiquant/dataset.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target_lag1'] = compute_lag1(df, lag_default_value=dc.lag_default_value)
100%|██████████| 84152/84152 [00:02<00:00, 30041.11it/s]


pearsonr,▁
pearsonr,0.09401


100%|██████████| 84152/84152 [00:02<00:00, 29888.98it/s]


pearsonr,▁
pearsonr,0.08675


100%|██████████| 84152/84152 [00:02<00:00, 30136.44it/s]


...

pearsonr,▁
pearsonr,0.15612


100%|██████████| 64668/64668 [00:02<00:00, 28310.24it/s]


pearsonr,▁
pearsonr,0.15594


100%|██████████| 64978/64978 [00:02<00:00, 29223.96it/s]


pearsonr,▁
pearsonr,0.15356
